In [5]:
from espnet.utils.io_utils import LoadInputsAndTargets
import json

In [4]:
load_tr = LoadInputsAndTargets(
        mode='asr', load_output=True,
        preprocess_args={'train': True}  # Switch the mode of preprocessing
    )

In [16]:
load_tr([data["utts"]["FDHC0_SI1559"], data["utts"]["FDHC0_SI2189"]])

ValueError: too many values to unpack (expected 2)